In [102]:
import pandas as pd
import numpy as np 
import numpy.random as npr
from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta
import io
import re
from scipy.interpolate import interp1d
from datetime import datetime
from datetime import timezone
from math import sqrt
from scipy.stats import norm

In [2]:
capnormal_data=pd.read_csv("HA_capNormalVol.csv")
swaps_data=pd.read_csv("HA_swaps.csv")
swaption_data=pd.read_csv("HA_swaptionNormalVol.csv")

In [3]:
def schedule(start, end, freq=6):            
    def advance(dt, months):
        try:
            return date(dt.year, dt.month + months, dt.day)
        except:
            return date(dt.year + 1, dt.month - months, dt.day)
    dates = [start]
    while dates[-1] <= end:
        dates.append(advance(dates[-1], 6))
    return dates[:-1]

In [4]:
#Exercise 1
#creating schedule of dates
today_date = date(2007,1,15)
maturity = date(2022,1,19)
start_date = date(2007,1,19)

In [5]:
#Exercise 1
swaps = swaps_data.rename(columns={'Unnamed: 0': 'Date'}) #changing name of columns
swaps['Date'] = pd.to_datetime(swaps['Date']) #setting the Date columns to a datetime columns
swaps.set_index('Date', inplace=True) #setting the column as the index
swaps

,Deposit 6M,FRA 1X7,FRA 2X8,FRA 3X9,FRA 4X10,FRA 5X11,FRA 6X12,FRA 7X13,FRA 8X14,FRA 9X15,...,Swap 9Y,Swap 10Y,Swap 11Y,Swap 12Y,Swap 15Y,Swap 20Y,Swap 25Y,Swap 30Y,Swap 40Y,Swap 50Y
Date,,,,,,,,,,,,,,,,,,,,,
2007-01-02,3.857,3.9285,3.9770,4.0155,4.0455,4.0755,4.0975,4.0965,4.1010,4.1005,...,4.1520,4.1720,4.1870,4.2060,4.2445,4.2840,4.2750,4.2590,4.2160,4.1740
2007-01-03,3.860,3.9170,3.9720,4.0145,4.0410,4.0670,4.0820,4.0870,4.0920,4.0890,...,4.1505,4.1715,4.1885,4.2045,4.2580,4.2845,4.2950,4.2635,4.2205,4.1785
2007-01-04,3.865,3.9315,3.9865,4.0225,4.0435,4.0720,4.0915,4.0975,4.1025,4.0985,...,4.1445,4.1625,4.1815,4.1985,4.2455,4.2755,4.2780,4.2565,4.2145,4.1725
2007-01-05,3.864,3.9415,3.9950,4.0410,4.0710,4.0950,4.1220,4.1305,4.1365,4.1325,...,4.1875,4.2055,4.2235,4.2405,4.2885,4.3175,4.3290,4.2970,4.2550,4.2150
2007-01-08,3.874,3.9435,4.0015,4.0435,4.0695,4.0985,4.1225,4.1215,4.1275,4.1235,...,4.1860,4.2060,4.2260,4.2420,4.2885,4.3200,4.3210,4.3000,4.2590,4.2200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-04-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.1620,0.2107,NaN,0.1346,0.0610,NaN
2020-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.1935,NaN,NaN,NaN,-0.0085,0.0341,-0.0012,-0.0610,-0.1590,NaN
2019-12-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.3050,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5760,0.5300,NaN


In [6]:
swaps_v1=swaps.loc['2007-01-15 00:00:00'].transpose()
swaps_v1.columns=['par_rate']
swaps_v1.reset_index(inplace=True)
swaps_v1

,index,par_rate
0,Deposit 6M,3.8800
1,FRA 1X7,3.9520
2,FRA 2X8,4.0260
3,FRA 3X9,4.0659
4,FRA 4X10,4.1048
5,FRA 5X11,4.1386
6,FRA 6X12,4.1543
7,FRA 7X13,4.1606
8,FRA 8X14,4.1689
9,FRA 9X15,4.1675


In [7]:
swaps_v1[['instrument', 'maturity']] = swaps_v1['index'].str.split(' ', expand=True) #splitting the index column to have a columns with only the name of the product

In [8]:
swaps_v1["maturity"] = swaps_v1["maturity"].apply(lambda x: x + "M" if "X" in x else x) #adding a M to the FRAs maturities

In [9]:
swaps_v1['maturity_fra'] = swaps_v1['maturity'].str.split('X').str[1] #getting the maturity of the fra

In [10]:
swaps_v1.loc[swaps_v1["instrument"] == "FRA", "maturity"] = swaps_v1["maturity_fra"] #getting the new maturity

In [11]:
def extract_number(string):
    # Use a regular expression to extract the numeric value from the string
    number = re.findall(r'\d+', string)
    # If the string contains the character 'M', return the number as a float
    # (dividing the number by 12 to convert from months to years)
    if 'M' in string:
        return float(number[0]) / 12
    # Otherwise, return the number as an integer
    else:
        return int(number[0])

In [12]:
swaps_v1["year_frac"] = swaps_v1["maturity"].apply(extract_number)

In [13]:
swaps_v1.set_index('index',inplace=True)

In [14]:
forward=swaps_v1.loc['FRA 6X12', 'par_rate']

In [15]:
one_year_rate=(((1+swaps_v1.loc['Deposit 6M', 'par_rate'])**swaps_v1.loc['Deposit 6M', 'year_frac'])*(1+forward)**(1-swaps_v1.loc['Deposit 6M', 'year_frac']))-1
one_year_rate

4.015275067232105

In [16]:
yield_curve = swaps_v1[swaps_v1.index.str.contains("FRA") == False]

In [17]:
one_year_rate_frame = pd.DataFrame([[one_year_rate,'-','1Y','-',1.0]],
     index=['Swap 1Y'],
     columns=['par_rate', 'instrument','maturity','maturity_fra','year_frac'])
one_year_rate_frame

,par_rate,instrument,maturity,maturity_fra,year_frac
Swap 1Y,4.015275,-,1Y,-,1.0


In [18]:
yield_curve=yield_curve.append(one_year_rate_frame)

In [19]:
yield_curve.set_index('year_frac',inplace=True)

In [20]:
yield_curve.loc[0.5, 'df'] = 1 / (1 + (yield_curve.loc[0.5, 'par_rate']/100) * 0.5)
yield_curve.loc[0.5, 'zero_rate'] = yield_curve.loc[0.5, 'par_rate']/100

In [21]:
yield_curve.loc[1, 'df'] = 1 / (1 + yield_curve.loc[1, 'par_rate']/100 * 1)
yield_curve.loc[1, 'zero_rate'] = yield_curve.loc[1, 'par_rate']/100

In [22]:
yield_curve=yield_curve.sort_index()

In [23]:
yield_curve

,par_rate,instrument,maturity,maturity_fra,df,zero_rate
year_frac,,,,,,
0.5,3.880000,Deposit,6M,NaN,0.980969,0.038800
1.0,4.015275,-,1Y,-,0.961397,0.040153
2.0,4.169000,Swap,2Y,NaN,NaN,NaN
3.0,4.172500,Swap,3Y,NaN,NaN,NaN
4.0,4.175500,Swap,4Y,NaN,NaN,NaN
5.0,4.184500,Swap,5Y,NaN,NaN,NaN
6.0,4.202500,Swap,6Y,NaN,NaN,NaN
7.0,4.218500,Swap,7Y,NaN,NaN,NaN
8.0,4.242500,Swap,8Y,NaN,NaN,NaN


In [24]:
date(start_date.year,start_date.month,start_date.day)

datetime.date(2007, 1, 19)

In [25]:
yield_curve["Tenor"]= yield_curve.index
yield_curve['maturity'] = [date(start_date.year,start_date.month+6,start_date.day)] + [date(2007 +int(n), 1, 19) for n in yield_curve.index[1:]]
yield_curve['days'] = yield_curve.maturity.apply(lambda x: (x - start_date).days)
yield_curve

,par_rate,instrument,maturity,maturity_fra,df,zero_rate,Tenor,days
year_frac,,,,,,,,
0.5,3.880000,Deposit,2007-07-19,NaN,0.980969,0.038800,0.5,181
1.0,4.015275,-,2008-01-19,-,0.961397,0.040153,1.0,365
2.0,4.169000,Swap,2009-01-19,NaN,NaN,NaN,2.0,731
3.0,4.172500,Swap,2010-01-19,NaN,NaN,NaN,3.0,1096
4.0,4.175500,Swap,2011-01-19,NaN,NaN,NaN,4.0,1461
5.0,4.184500,Swap,2012-01-19,NaN,NaN,NaN,5.0,1826
6.0,4.202500,Swap,2013-01-19,NaN,NaN,NaN,6.0,2192
7.0,4.218500,Swap,2014-01-19,NaN,NaN,NaN,7.0,2557
8.0,4.242500,Swap,2015-01-19,NaN,NaN,NaN,8.0,2922


In [26]:
for idx, row in yield_curve.loc[2:].iterrows():
    prev_dfs = yield_curve.loc[1:idx-1].df.sum()
    yield_curve.loc[idx, 'df'] = (1 - row.par_rate/100 * prev_dfs) / (1+row.par_rate/100)

In [27]:

for a in yield_curve.index[2:]:
    yield_curve.loc[a,'zero_rate']=((1/yield_curve.loc[a,'df'])**(1/a))-1


In [28]:
start_date = date(2007,1,19)
# Add six months and keep the same day
loan_dates=[start_date]
for i in range(30):
  resulting_date = start_date + relativedelta(months=+6)
  if resulting_date.weekday()==5:
    resulting_date=resulting_date+timedelta(days=2)
  if resulting_date.weekday()==6:
    resulting_date=resulting_date+timedelta(days=1)
  loan_dates.append(resulting_date)
  start_date=date(resulting_date.year,resulting_date.month,19)
start_date = date(2007,1,19)
loan_dates

[datetime.date(2007, 1, 19),
 datetime.date(2007, 7, 19),
 datetime.date(2008, 1, 21),
 datetime.date(2008, 7, 21),
 datetime.date(2009, 1, 19),
 datetime.date(2009, 7, 20),
 datetime.date(2010, 1, 19),
 datetime.date(2010, 7, 19),
 datetime.date(2011, 1, 19),
 datetime.date(2011, 7, 19),
 datetime.date(2012, 1, 19),
 datetime.date(2012, 7, 19),
 datetime.date(2013, 1, 21),
 datetime.date(2013, 7, 19),
 datetime.date(2014, 1, 20),
 datetime.date(2014, 7, 21),
 datetime.date(2015, 1, 19),
 datetime.date(2015, 7, 20),
 datetime.date(2016, 1, 19),
 datetime.date(2016, 7, 19),
 datetime.date(2017, 1, 19),
 datetime.date(2017, 7, 19),
 datetime.date(2018, 1, 19),
 datetime.date(2018, 7, 19),
 datetime.date(2019, 1, 21),
 datetime.date(2019, 7, 19),
 datetime.date(2020, 1, 20),
 datetime.date(2020, 7, 20),
 datetime.date(2021, 1, 19),
 datetime.date(2021, 7, 19),
 datetime.date(2022, 1, 19)]

In [29]:
nr_days=[]
for a in range(0,len(loan_dates)):
    if a ==0:
      pass
    else:
      nr_days.append((loan_dates[a]-today_date).days/365)
nr_days

[0.5068493150684932,
 1.0164383561643835,
 1.515068493150685,
 2.0136986301369864,
 2.5123287671232877,
 3.0136986301369864,
 3.5095890410958903,
 4.013698630136986,
 4.50958904109589,
 5.013698630136986,
 5.512328767123288,
 6.021917808219178,
 6.512328767123288,
 7.019178082191781,
 7.517808219178082,
 8.016438356164384,
 8.515068493150684,
 9.016438356164384,
 9.515068493150684,
 10.01917808219178,
 10.515068493150684,
 11.01917808219178,
 11.515068493150684,
 12.024657534246575,
 12.515068493150684,
 13.021917808219179,
 13.520547945205479,
 14.021917808219179,
 14.517808219178082,
 15.021917808219179]

In [30]:
y=np.array(yield_curve.zero_rate.values)
x=np.array(yield_curve.index.values)
y_interp = interp1d(x,y)
 
# Finding the interpolation
interpolated_rates=[]
for a in nr_days:
  interpolate_x=a
  interpolated_rates.append(float(y_interp(interpolate_x)))
interpolated_rates

[0.038818530831127684,
 0.0401785480772134,
 0.04096106935894772,
 0.04172244627297853,
 0.04173531257030465,
 0.04174825530203538,
 0.04176125871019032,
 0.04177540709209084,
 0.041822056504942834,
 0.04187087887849022,
 0.041968737505568496,
 0.0420683090732121,
 0.04215475890475419,
 0.04224594754610628,
 0.04238171355993787,
 0.04251715209004331,
 0.04264298443084846,
 0.04276977762419703,
 0.04290378376868447,
 0.04303887093294358,
 0.04316201576346114,
 0.043286799221674215,
 0.04339953792666865,
 0.043465482029131246,
 0.04258434181141895,
 0.04167366616741463,
 0.04077775823655632,
 0.03987692773464934,
 0.038985942374839704,
 0.03808843304905318]

In [31]:
notional=1748653.15
installemnts=np.full(30,1748653.15)
int_pay=installemnts*interpolated_rates
int_pay[len(int_pay)-1]=notional*(1+interpolated_rates[len(interpolated_rates)-1])
int_pay

array([ 67880.14621622, 70258.34465765, 71626.70296189,
 72958.08710095, 72980.5857923 , 73003.21814091,
 73025.95659154, 73050.69720412, 73132.27084685,
 73217.64424414, 73388.76504064, 73562.88117605,
 73714.05194629, 73873.50925123, 74110.91691898,
 74347.75193128, 74567.7890504 , 74789.50636735,
 75023.83663403, 75260.05722934, 75475.39482513,
 75693.5978124 , 75890.73870401, 76006.05206651,
 74465.24344921, 72872.7876157 , 71306.15539029,
 69730.91529552, 68172.89093948, 1815256.60842979])

In [32]:
#Exercise 2
#The company is Party A
swap_notional=1748653.15
start_swap=loan_dates[0]
maturity_swap=loan_dates[-1]
#Party A Receives Euribor 6m, half yearly, act/360
#Party A Pays xxxxxxxx, half yearly, act/360

In [33]:
#TESTE
yield_curve_swap = yield_curve.copy()
yield_curve_swap

,par_rate,instrument,maturity,maturity_fra,df,zero_rate,Tenor,days
year_frac,,,,,,,,
0.5,3.880000,Deposit,2007-07-19,NaN,0.980969,0.038800,0.5,181
1.0,4.015275,-,2008-01-19,-,0.961397,0.040153,1.0,365
2.0,4.169000,Swap,2009-01-19,NaN,0.921502,0.041722,2.0,731
3.0,4.172500,Swap,2010-01-19,NaN,0.884529,0.041748,3.0,1096
4.0,4.175500,Swap,2011-01-19,NaN,0.848996,0.041774,4.0,1461
5.0,4.184500,Swap,2012-01-19,NaN,0.814584,0.041868,5.0,1826
6.0,4.202500,Swap,2013-01-19,NaN,0.780967,0.042064,6.0,2192
7.0,4.218500,Swap,2014-01-19,NaN,0.748555,0.042241,7.0,2557
8.0,4.242500,Swap,2015-01-19,NaN,0.716718,0.042513,8.0,2922


In [34]:
def getDF(dt):
    x = (dt - start_date).days
    return np.interp(x, yield_curve_swap.days, yield_curve_swap.df)

def getFwd(start, end):
    DF0 = getDF(start)
    DF1 = getDF(end)
    n = (end - start).days / 360
    return (DF0 / DF1 - 1) / n

In [35]:
#get floating leg
pd.options.display.float_format = '{:,.5f}'.format
float_leg = pd.DataFrame({
    'notional': swap_notional,
    'start': loan_dates[:-1],
    'end': loan_dates[1:]
})

float_leg['dcf'] = float_leg.apply(lambda x: (x.end - x.start).days / 360, axis=1)
float_leg['fwd'] = float_leg.apply(lambda row: getFwd(row.start, row.end), axis=1)
float_leg.loc[0, 'fwd'] = yield_curve_swap.loc[0.5, 'par_rate']/100
float_leg['cf'] = float_leg.notional * float_leg.dcf * float_leg.fwd
float_leg['df'] = float_leg.end.apply(getDF)
float_leg['pv'] = float_leg.cf * float_leg.df


float_leg_pv = float_leg.pv.sum()
print(f"PV of float leg is: {float_leg_pv:,.2f}")
float_leg

PV of float leg is: 751,154.04


,notional,start,end,dcf,fwd,cf,df,pv
0,"1,748,653.15000",2007-01-19,2007-07-19,0.50278,0.03880,"34,112.33706",0.98097,"33,463.15191"
1,"1,748,653.15000",2007-07-19,2008-01-21,0.51667,0.03985,"36,003.43733",0.96118,"34,605.75683"
2,"1,748,653.15000",2008-01-21,2008-07-21,0.50556,0.04169,"36,852.68842",0.94134,"34,690.93149"
3,"1,748,653.15000",2008-07-21,2009-01-19,0.50556,0.04258,"37,646.07566",0.92150,"34,690.93149"
4,"1,748,653.15000",2009-01-19,2009-07-20,0.50556,0.04038,"35,698.16367",0.90307,"32,237.80296"
5,"1,748,653.15000",2009-07-20,2010-01-19,0.50833,0.04123,"36,646.54485",0.88453,"32,414.93374"
6,"1,748,653.15000",2010-01-19,2010-07-19,0.50278,0.04043,"35,542.35504",0.86691,"30,811.97511"
7,"1,748,653.15000",2010-07-19,2011-01-19,0.51111,0.04128,"36,893.77046",0.84900,"31,322.67083"
8,"1,748,653.15000",2011-01-19,2011-07-19,0.50278,0.04080,"35,868.12115",0.83193,"29,839.82821"
9,"1,748,653.15000",2011-07-19,2012-01-19,0.51111,0.04167,"37,239.12433",0.81458,"30,334.41100"


In [36]:
fixed_leg = pd.DataFrame({
    'notional': swap_notional,
    'start': loan_dates[::1][:-1],
    'end': loan_dates[::1][1:]
})

fixed_leg['dcf'] = 1
fixed_leg['df'] = fixed_leg.end.apply(getDF)
fixed_leg

,notional,start,end,dcf,df
0,"1,748,653.15000",2007-01-19,2007-07-19,1,0.98097
1,"1,748,653.15000",2007-07-19,2008-01-21,1,0.96118
2,"1,748,653.15000",2008-01-21,2008-07-21,1,0.94134
3,"1,748,653.15000",2008-07-21,2009-01-19,1,0.92150
4,"1,748,653.15000",2009-01-19,2009-07-20,1,0.90307
5,"1,748,653.15000",2009-07-20,2010-01-19,1,0.88453
6,"1,748,653.15000",2010-01-19,2010-07-19,1,0.86691
7,"1,748,653.15000",2010-07-19,2011-01-19,1,0.84900
8,"1,748,653.15000",2011-01-19,2011-07-19,1,0.83193
9,"1,748,653.15000",2011-07-19,2012-01-19,1,0.81458


In [37]:
#getting swap rate with matrix algebra
fixed_rate = float_leg_pv / (fixed_leg.notional *  fixed_leg.dcf * fixed_leg.df).sum()
print(f"Fair fixed rate is: {fixed_rate * 100:.4f}%")

Fair fixed rate is: 1.9358%


In [38]:
from scipy.optimize import root
def swapPv(rate):
    fxd = fixed_leg.copy()
    fixed_leg_pv = (fxd.notional * fxd.dcf * fxd.df * rate).sum()
    return fixed_leg_pv - float_leg_pv


res = root(swapPv, 0)
fixed_rate1 = res['x'][0]

print(f"Fixed rate Matrix Algebra:\t {fixed_rate * 100:.4f}%")
print(f"Fixed rate Optimization:\t {fixed_rate1 * 100:.4f}%")

Fixed rate Matrix Algebra:	 1.9358%
Fixed rate Optimization:	 1.9358%


In [39]:
fixed_leg['rate'] = fixed_rate
fixed_leg['cashflow'] = fixed_leg.notional * fixed_leg.dcf * fixed_leg.rate
fixed_leg['pv'] = fixed_leg.cashflow * fixed_leg.df


fixed_leg_pv = fixed_leg.pv.sum()
print(f"Present Value of Fixed Leg is: {fixed_leg_pv:,.2f}")

fixed_leg

Present Value of Fixed Leg is: 751,154.04


,notional,start,end,dcf,df,rate,cashflow,pv
0,"1,748,653.15000",2007-01-19,2007-07-19,1,0.98097,0.01936,"33,850.14008","33,205.94475"
1,"1,748,653.15000",2007-07-19,2008-01-21,1,0.96118,0.01936,"33,850.14008","32,536.05220"
2,"1,748,653.15000",2008-01-21,2008-07-21,1,0.94134,0.01936,"33,850.14008","31,864.51086"
3,"1,748,653.15000",2008-07-21,2009-01-19,1,0.92150,0.01936,"33,850.14008","31,192.96951"
4,"1,748,653.15000",2009-01-19,2009-07-20,1,0.90307,0.01936,"33,850.14008","30,568.91542"
5,"1,748,653.15000",2009-07-20,2010-01-19,1,0.88453,0.01936,"33,850.14008","29,941.43247"
6,"1,748,653.15000",2010-01-19,2010-07-19,1,0.86691,0.01936,"33,850.14008","29,344.97932"
7,"1,748,653.15000",2010-07-19,2011-01-19,1,0.84900,0.01936,"33,850.14008","28,738.64021"
8,"1,748,653.15000",2011-01-19,2011-07-19,1,0.83193,0.01936,"33,850.14008","28,161.00572"
9,"1,748,653.15000",2011-07-19,2012-01-19,1,0.81458,0.01936,"33,850.14008","27,573.79718"


In [40]:
#Question 2b)
pv01 = (fixed_leg.notional * fixed_leg.dcf * 0.0001 * fixed_leg.df).sum()

print(f"Present value of 1 bps on the fixed leg is: EUR {pv01:,.2f}")

Present value of 1 bps on the fixed leg is: EUR 3,880.36


In [41]:
#question 2c
fixed_leg_2c = pd.DataFrame({
    'notional': swap_notional,
    'start': loan_dates[::2][:-1],
    'end': loan_dates[::2][1:]
})

fixed_leg_2c['dcf'] = 1
fixed_leg_2c['df'] = fixed_leg.end.apply(getDF)
fixed_leg_2c

,notional,start,end,dcf,df
0,"1,748,653.15000",2007-01-19,2008-01-21,1,0.98097
1,"1,748,653.15000",2008-01-21,2009-01-19,1,0.96118
2,"1,748,653.15000",2009-01-19,2010-01-19,1,0.94134
3,"1,748,653.15000",2010-01-19,2011-01-19,1,0.92150
4,"1,748,653.15000",2011-01-19,2012-01-19,1,0.90307
5,"1,748,653.15000",2012-01-19,2013-01-21,1,0.88453
6,"1,748,653.15000",2013-01-21,2014-01-20,1,0.86691
7,"1,748,653.15000",2014-01-20,2015-01-19,1,0.84900
8,"1,748,653.15000",2015-01-19,2016-01-19,1,0.83193
9,"1,748,653.15000",2016-01-19,2017-01-19,1,0.81458


In [42]:
#question 2c

fixed_leg_2c['rate'] = fixed_rate+0.0005
fixed_leg_2c['cashflow'] = fixed_leg_2c.notional * fixed_leg_2c.dcf * fixed_leg_2c.rate
fixed_leg_2c['pv'] = fixed_leg_2c.cashflow * fixed_leg_2c.df


fixed_leg_pv_2c = fixed_leg_2c.pv.sum()
print(f"Present Value of Fixed Leg, with the fee, is: {fixed_leg_pv_2c:,.2f}")

Present Value of Fixed Leg, with the fee, is: 443,765.43


In [43]:
#Other way to calculate querstion 2c
pv05 = (fixed_leg.notional * fixed_leg.dcf * 0.0005 * fixed_leg.df).sum()
result_pv05 = fixed_leg_pv + pv05
print(f"The price of this swap considering that the counterparty charges 5bps is: {result_pv05:.3f}")

The price of this swap considering that the counterparty charges 5bps is: 770555.849


In [44]:
#The price of the swap with a 5bps is  3.7953% + 0.0005

In [45]:
#question 2d)
#The market value of this swap 0
market_value = fixed_leg_pv - float_leg_pv
market_value

2.3283064365386963e-10

In [46]:
#Market value with 5bps fee:
market_value_fee = fixed_leg_pv_2c - float_leg_pv
market_value_fee

-307388.6082100409

In [47]:
#Qustion 2e)
beg_date=date(2007,1,31)
last_date=date(2020,11,30)
delta=(last_date-beg_date).days

In [48]:
hist_dates=pd.date_range(beg_date,last_date,freq='d')
hist_dates

DatetimeIndex(['2007-01-31', '2007-02-01', '2007-02-02', '2007-02-03',
               '2007-02-04', '2007-02-05', '2007-02-06', '2007-02-07',
               '2007-02-08', '2007-02-09',
               ...
               '2020-11-21', '2020-11-22', '2020-11-23', '2020-11-24',
               '2020-11-25', '2020-11-26', '2020-11-27', '2020-11-28',
               '2020-11-29', '2020-11-30'],
              dtype='datetime64[ns]', length=5053, freq='D')

In [49]:
def getDF2(dt):
    x = (dt - start_date).days
    return np.interp(x, yield_curve_hist.days, yield_curve_hist.df)

def getFwd2(start, end):
    DF0 = getDF2(start)
    DF1 = getDF2(end)
    n = (end - start).days / 360
    return (DF0 / DF1 - 1) / n

In [50]:
market_value_hist=[]
for b in hist_dates:
    try:
        swaps_hist=swaps.loc[str(b)].transpose()
        swaps_hist.columns=['par_rate']
        swaps_hist.reset_index(inplace=True)
        swaps_hist[['instrument', 'maturity']] = swaps_hist['index'].str.split(' ', expand=True) #splitting the index column to have a columns with only the name of the product
        swaps_hist["maturity"] = swaps_hist["maturity"].apply(lambda x: x + "M" if "X" in x else x) #adding a M to the FRAs maturities
        swaps_hist['maturity_fra'] = swaps_hist['maturity'].str.split('X').str[1] #getting the maturity of the fra
        swaps_hist.loc[swaps_hist["instrument"] == "FRA", "maturity"] = swaps_hist["maturity_fra"] #getting the new maturity
        swaps_hist["year_frac"] = swaps_hist["maturity"].apply(extract_number)
        swaps_hist.set_index('index',inplace=True)
        forward_hist=swaps_hist.loc['FRA 6X12', 'par_rate']
        one_year_rate_hist=(((1+swaps_hist.loc['Deposit 6M', 'par_rate'])**swaps_hist.loc['Deposit 6M', 'year_frac'])*(1+forward_hist)**(1-swaps_hist.loc['Deposit 6M', 'year_frac']))-1
        yield_curve_hist = swaps_hist[swaps_hist.index.str.contains("FRA") == False]
        one_year_rate_frame_hist = pd.DataFrame([[one_year_rate_hist,'-','1Y','-',1.0]],
            index=['Swap 1Y'],
            columns=['par_rate', 'instrument','maturity','maturity_fra','year_frac'])
        yield_curve_hist=yield_curve_hist.append(one_year_rate_frame_hist)
        yield_curve_hist.set_index('year_frac',inplace=True)
        yield_curve_hist.loc[0.5, 'df'] = 1 / (1 + (yield_curve_hist.loc[0.5, 'par_rate']/100) * 0.5)
        yield_curve_hist.loc[0.5, 'zero_rate'] = yield_curve_hist.loc[0.5, 'par_rate']/100
        yield_curve_hist.loc[1, 'df'] = 1 / (1 + yield_curve_hist.loc[1, 'par_rate']/100 * 1)
        yield_curve_hist.loc[1, 'zero_rate'] = yield_curve_hist.loc[1, 'par_rate']/100
        yield_curve_hist=yield_curve_hist.sort_index()
        yield_curve_hist["Tenor"]= yield_curve_hist.index
        yield_curve_hist['maturity'] = [b+relativedelta(months=+6)] + [date(b.year +int(n), b.month, b.day) for n in yield_curve_hist.index[1:]]
        for n in range(0,len(yield_curve_hist['maturity'])):
            yield_curve_hist.iloc[n,2]=date(yield_curve_hist.iloc[n,2].year,yield_curve_hist.iloc[n,2].month,yield_curve_hist.iloc[n,2].day)
        yield_curve_hist['days'] = yield_curve_hist.maturity.apply(lambda x: (x - date(b.year,b.month,b.day)).days)
        idx_hist=0
        row_hist=0
        for idx_hist, row_hist in yield_curve_hist.loc[2:].iterrows():
            prev_dfs = yield_curve_hist.loc[1:idx_hist-1].df.sum()
            yield_curve_hist.loc[idx_hist, 'df'] = (1 - row_hist.par_rate/100 * prev_dfs) / (1+row_hist.par_rate/100)
        for a in yield_curve_hist.index[2:]:
            yield_curve_hist.loc[a,'zero_rate']=((1/yield_curve_hist.loc[a,'df'])**(1/a))-1
        b = date(b.year,b.month,b.day)
        
        pd.options.display.float_format = '{:,.5f}'.format
        float_leg_hist = pd.DataFrame({'notional': swap_notional,'start': loan_dates[:-1],'end': loan_dates[1:]})
        float_leg_hist['dcf'] = float_leg_hist.apply(lambda x: (x.end - x.start).days / 360, axis=1)
        float_leg_hist['fwd'] = float_leg_hist.apply(lambda row: getFwd2(row.start, row.end), axis=1)
        float_leg_hist.loc[0, 'fwd'] = yield_curve_hist.loc[0.5, 'par_rate']/100
        float_leg_hist['cf'] = float_leg_hist.notional * float_leg_hist.dcf * float_leg_hist.fwd
        float_leg_hist['df'] = float_leg_hist.end.apply(getDF)
        float_leg_hist['pv'] = float_leg_hist.cf * float_leg_hist.df
        for n in range(0,len(float_leg_hist['notional'])):
            if  b>=float_leg_hist.iloc[n,2]:
                float_leg_hist.iloc[n,7]=0
        float_leg_hist_pv = float_leg_hist.pv.sum()
        
        
        fixed_leg_hist = pd.DataFrame({'notional': swap_notional,'start': loan_dates[::1][:-1],'end': loan_dates[::1][1:]})
        fixed_leg_hist['dcf'] = 1
        fixed_leg_hist['df'] = fixed_leg_hist.end.apply(getDF)
        fixed_leg_hist['rate'] = fixed_rate
        fixed_leg_hist['cashflow'] = fixed_leg_hist.notional * fixed_leg_hist.dcf * fixed_leg_hist.rate
        fixed_leg_hist['pv'] = fixed_leg_hist.cashflow * fixed_leg_hist.df
        for n in range(0,len(fixed_leg_hist['notional'])):
            if  b>=fixed_leg_hist.iloc[n,2]:
                fixed_leg_hist.iloc[n,7]=0
        fixed_leg_hist_pv = fixed_leg_hist.pv.sum()
        
        mk_value_hist=fixed_leg_hist_pv-float_leg_hist_pv
        
        
        market_value_hist.append(mk_value_hist)
    
    
    except:
        continue


,notional,start,end,dcf,df,rate,cashflow,pv
0,"1,748,653.15000",2007-01-19,2007-07-19,1,0.98097,0.01936,"33,850.14008",0.00000
1,"1,748,653.15000",2007-07-19,2008-01-21,1,0.96118,0.01936,"33,850.14008",0.00000
2,"1,748,653.15000",2008-01-21,2008-07-21,1,0.94134,0.01936,"33,850.14008",0.00000
3,"1,748,653.15000",2008-07-21,2009-01-19,1,0.92150,0.01936,"33,850.14008",0.00000
4,"1,748,653.15000",2009-01-19,2009-07-20,1,0.90307,0.01936,"33,850.14008",0.00000
5,"1,748,653.15000",2009-07-20,2010-01-19,1,0.88453,0.01936,"33,850.14008",0.00000
6,"1,748,653.15000",2010-01-19,2010-07-19,1,0.86691,0.01936,"33,850.14008",0.00000
7,"1,748,653.15000",2010-07-19,2011-01-19,1,0.84900,0.01936,"33,850.14008",0.00000
8,"1,748,653.15000",2011-01-19,2011-07-19,1,0.83193,0.01936,"33,850.14008",0.00000
9,"1,748,653.15000",2011-07-19,2012-01-19,1,0.81458,0.01936,"33,850.14008",0.00000


In [51]:
len(market_value_hist)

3606

In [52]:
available_dates=swaps.index
available_dates=pd.DataFrame(available_dates)

available_dates = available_dates.loc[(available_dates['Date'] >= '2007-01-31')
                     & (available_dates['Date'] <= '2020-11-30')]
available_dates

,Date
21,2007-01-31
22,2007-02-01
23,2007-02-02
24,2007-02-05
25,2007-02-06
...,...
3640,2020-04-10
3641,2020-05-01
3642,2019-12-26
3643,2020-01-01


In [145]:
#alternative 2
#pv_cap = max(euribor - K;0)
#k =6 %
float_leg_alt2=float_leg
float_leg_alt2["cap_pv"] =1
for n in range(0,len(float_leg_alt2["fwd"])):
    float_leg_alt2.iloc[n,8]=max(float_leg_alt2["fwd"][n]- 0.06, 0)
float_leg_alt2

,notional,start,end,dcf,fwd,cf,df,pv,cap_pv,price-cap,tenor
0,"1,748,653.15000",2007-01-19,2007-07-19,0.50278,0.03880,"34,112.33706",0.98097,"33,463.15191",0,-3.75913,0.50000
1,"1,748,653.15000",2007-07-19,2008-01-21,0.51667,0.03985,"36,003.43733",0.96118,"34,605.75683",0,-3.59652,1.00000
2,"1,748,653.15000",2008-01-21,2008-07-21,0.50556,0.04169,"36,852.68842",0.94134,"34,690.93149",0,-3.13440,1.50000
3,"1,748,653.15000",2008-07-21,2009-01-19,0.50556,0.04258,"37,646.07566",0.92150,"34,690.93149",0,-2.92113,2.00000
4,"1,748,653.15000",2009-01-19,2009-07-20,0.50556,0.04038,"35,698.16367",0.90307,"32,237.80296",0,-3.22483,2.50000
5,"1,748,653.15000",2009-07-20,2010-01-19,0.50833,0.04123,"36,646.54485",0.88453,"32,414.93374",0,-3.04023,3.00000
6,"1,748,653.15000",2010-01-19,2010-07-19,0.50278,0.04043,"35,542.35504",0.86691,"30,811.97511",0,-3.07278,3.50000
7,"1,748,653.15000",2010-07-19,2011-01-19,0.51111,0.04128,"36,893.77046",0.84900,"31,322.67083",0,-2.92713,4.00000
8,"1,748,653.15000",2011-01-19,2011-07-19,0.50278,0.04080,"35,868.12115",0.83193,"29,839.82821",0,-2.89480,4.50000
9,"1,748,653.15000",2011-07-19,2012-01-19,0.51111,0.04167,"37,239.12433",0.81458,"30,334.41100",0,-2.75278,5.00000


In [146]:
float_leg_alt2['tenor'] = [x * 0.5 for x in range(1, len(float_leg_alt2) + 1)]
float_leg_alt2

,notional,start,end,dcf,fwd,cf,df,pv,cap_pv,price-cap,tenor
0,"1,748,653.15000",2007-01-19,2007-07-19,0.50278,0.03880,"34,112.33706",0.98097,"33,463.15191",0,-3.75913,0.50000
1,"1,748,653.15000",2007-07-19,2008-01-21,0.51667,0.03985,"36,003.43733",0.96118,"34,605.75683",0,-3.59652,1.00000
2,"1,748,653.15000",2008-01-21,2008-07-21,0.50556,0.04169,"36,852.68842",0.94134,"34,690.93149",0,-3.13440,1.50000
3,"1,748,653.15000",2008-07-21,2009-01-19,0.50556,0.04258,"37,646.07566",0.92150,"34,690.93149",0,-2.92113,2.00000
4,"1,748,653.15000",2009-01-19,2009-07-20,0.50556,0.04038,"35,698.16367",0.90307,"32,237.80296",0,-3.22483,2.50000
5,"1,748,653.15000",2009-07-20,2010-01-19,0.50833,0.04123,"36,646.54485",0.88453,"32,414.93374",0,-3.04023,3.00000
6,"1,748,653.15000",2010-01-19,2010-07-19,0.50278,0.04043,"35,542.35504",0.86691,"30,811.97511",0,-3.07278,3.50000
7,"1,748,653.15000",2010-07-19,2011-01-19,0.51111,0.04128,"36,893.77046",0.84900,"31,322.67083",0,-2.92713,4.00000
8,"1,748,653.15000",2011-01-19,2011-07-19,0.50278,0.04080,"35,868.12115",0.83193,"29,839.82821",0,-2.89480,4.50000
9,"1,748,653.15000",2011-07-19,2012-01-19,0.51111,0.04167,"37,239.12433",0.81458,"30,334.41100",0,-2.75278,5.00000


In [140]:
capnormal_data[capnormal_data['Strike']=="6.00%"]

,Tenor,Strike,NormalVol,Int_Year
150,1Yr,6.00%,112.34000,1
151,2Yr,6.00%,111.57000,2
152,3Yr,6.00%,121.51000,3
153,4Yr,6.00%,109.08000,4
154,5Yr,6.00%,109.08000,5
155,6Yr,6.00%,109.18000,6
156,7Yr,6.00%,109.41000,7
157,8Yr,6.00%,109.69000,8
158,9Yr,6.00%,111.03000,9
159,10Yr,6.00%,111.24000,10


In [148]:
capnormal_data['Int_Year'] = capnormal_data['Tenor'].apply(lambda x: float(x[:-2]))
capnormal_data_st6=capnormal_data[capnormal_data['Strike']=="6.00%"]
float_leg_alt2 = float_leg_alt2.merge(capnormal_data_st6, left_on='tenor',right_on="Int_Year",how="left")
float_leg_alt2

,notional,start,end,dcf,fwd,cf,df,pv,cap_pv,price-cap,tenor,Tenor,Strike,NormalVol,Int_Year
0,"1,748,653.15000",2007-01-19,2007-07-19,0.50278,0.03880,"34,112.33706",0.98097,"33,463.15191",0,-3.75913,0.50000,NaN,NaN,NaN,NaN
1,"1,748,653.15000",2007-07-19,2008-01-21,0.51667,0.03985,"36,003.43733",0.96118,"34,605.75683",0,-3.59652,1.00000,1Yr,6.00%,112.34000,1.00000
2,"1,748,653.15000",2008-01-21,2008-07-21,0.50556,0.04169,"36,852.68842",0.94134,"34,690.93149",0,-3.13440,1.50000,NaN,NaN,NaN,NaN
3,"1,748,653.15000",2008-07-21,2009-01-19,0.50556,0.04258,"37,646.07566",0.92150,"34,690.93149",0,-2.92113,2.00000,2Yr,6.00%,111.57000,2.00000
4,"1,748,653.15000",2009-01-19,2009-07-20,0.50556,0.04038,"35,698.16367",0.90307,"32,237.80296",0,-3.22483,2.50000,NaN,NaN,NaN,NaN
5,"1,748,653.15000",2009-07-20,2010-01-19,0.50833,0.04123,"36,646.54485",0.88453,"32,414.93374",0,-3.04023,3.00000,3Yr,6.00%,121.51000,3.00000
6,"1,748,653.15000",2010-01-19,2010-07-19,0.50278,0.04043,"35,542.35504",0.86691,"30,811.97511",0,-3.07278,3.50000,NaN,NaN,NaN,NaN
7,"1,748,653.15000",2010-07-19,2011-01-19,0.51111,0.04128,"36,893.77046",0.84900,"31,322.67083",0,-2.92713,4.00000,4Yr,6.00%,109.08000,4.00000
8,"1,748,653.15000",2011-01-19,2011-07-19,0.50278,0.04080,"35,868.12115",0.83193,"29,839.82821",0,-2.89480,4.50000,NaN,NaN,NaN,NaN
9,"1,748,653.15000",2011-07-19,2012-01-19,0.51111,0.04167,"37,239.12433",0.81458,"30,334.41100",0,-2.75278,5.00000,5Yr,6.00%,109.08000,5.00000


In [120]:
def caplet_price(K,F,sigma,rf,start_date_cap,end_date_cap):
    P = np.exp(-rf*(end_date_cap-today_date).days/360)
    d1 = (np.log(F/K) + sigma**2 * (end_date_cap-today_date).days / 2) / (sigma * np.sqrt((end_date_cap-today_date).days))
    d2 = d1 - sigma
    caplet = (end_date_cap-start_date_cap).days * P * ( F * norm.cdf(d1) - K * norm.cdf(d2))
    return caplet

In [117]:
'''float_leg['dcf'] = float_leg.apply(lambda x: (x.end - x.start).days / 360, axis=1)
float_leg['fwd'] = float_leg.apply(lambda row: getFwd(row.start, row.end), axis=1)'''
len(float_leg_alt2)

30

In [121]:
float_leg_alt2['price-cap']=float_leg_alt2.apply(lambda row:caplet_price(0.06,row.fwd,
                            capnormal_data.iloc[150,2],0.04,row.start,row.end),axis=1)
float_leg_alt2

TypeError: TypeError: unsupported operand type(s) for ** or pow(): 'list' and 'int'

In [63]:
capnormal_data.iloc[150,2]

112.34

In [61]:
capnormal_data

,Tenor,Strike,NormalVol
0,1Yr,-1.00%,284.76000
1,2Yr,-1.00%,190.45000
2,3Yr,-1.00%,158.78000
3,4Yr,-1.00%,140.91000
4,5Yr,-1.00%,129.31000
...,...,...,...
175,12Yr,7.00%,114.60000
176,15Yr,7.00%,114.91000
177,20Yr,7.00%,114.72000
178,25Yr,7.00%,113.97000


In [60]:
#pv_cap = max(euribor - K;0)
#k =6 %
float_leg_c=float_leg
float_leg_c["cap_pv"] =1
float_leg_c["floor_pv"] =1
for n in range(0,len(float_leg_a["fwd"])):
    float_leg_c.iloc[n,8]=max(float_leg_c["fwd"][n]- 0.06, 0)
    

float_leg_c

,notional,start,end,dcf,fwd,cf,df,pv,cap_pv,floor_pv
0,"1,748,653.15000",2007-01-19,2007-07-19,0.50278,0.03880,"34,112.33706",0.98097,"33,463.15191",0,1
1,"1,748,653.15000",2007-07-19,2008-01-21,0.51667,0.03985,"36,003.43733",0.96118,"34,605.75683",0,1
2,"1,748,653.15000",2008-01-21,2008-07-21,0.50556,0.04169,"36,852.68842",0.94134,"34,690.93149",0,1
3,"1,748,653.15000",2008-07-21,2009-01-19,0.50556,0.04258,"37,646.07566",0.92150,"34,690.93149",0,1
4,"1,748,653.15000",2009-01-19,2009-07-20,0.50556,0.04038,"35,698.16367",0.90307,"32,237.80296",0,1
5,"1,748,653.15000",2009-07-20,2010-01-19,0.50833,0.04123,"36,646.54485",0.88453,"32,414.93374",0,1
6,"1,748,653.15000",2010-01-19,2010-07-19,0.50278,0.04043,"35,542.35504",0.86691,"30,811.97511",0,1
7,"1,748,653.15000",2010-07-19,2011-01-19,0.51111,0.04128,"36,893.77046",0.84900,"31,322.67083",0,1
8,"1,748,653.15000",2011-01-19,2011-07-19,0.50278,0.04080,"35,868.12115",0.83193,"29,839.82821",0,1
9,"1,748,653.15000",2011-07-19,2012-01-19,0.51111,0.04167,"37,239.12433",0.81458,"30,334.41100",0,1


In [61]:
swaption_data

,Expiry,Tenor,NormalVol
0,1Mo,1Yr,294.95000
1,3Mo,1Yr,296.68000
2,6Mo,1Yr,298.57000
3,9Mo,1Yr,296.64000
4,1Yr,1Yr,293.30000
...,...,...,...
211,10Yr,30Yr,254.72000
212,12Yr,30Yr,250.95000
213,15Yr,30Yr,240.62000
214,20Yr,30Yr,221.72000


In [57]:
capnormal_data

,Tenor,Strike,NormalVol
0,1Yr,-1.00%,284.76000
1,2Yr,-1.00%,190.45000
2,3Yr,-1.00%,158.78000
3,4Yr,-1.00%,140.91000
4,5Yr,-1.00%,129.31000
...,...,...,...
175,12Yr,7.00%,114.60000
176,15Yr,7.00%,114.91000
177,20Yr,7.00%,114.72000
178,25Yr,7.00%,113.97000


In [95]:
#Question 2g)
b=date(2020,12,18)
swaps_hist_dez18=swaps.loc[str(b)].transpose()
swaps_hist_dez18.columns=['par_rate']
swaps_hist_dez18.reset_index(inplace=True)
swaps_hist_dez18[['instrument', 'maturity']] = swaps_hist_dez18['index'].str.split(' ', expand=True) #splitting the index column to have a columns with only the name of the product
swaps_hist_dez18["maturity"] = swaps_hist_dez18["maturity"].apply(lambda x: x + "M" if "X" in x else x) #adding a M to the FRAs maturities
swaps_hist_dez18['maturity_fra'] = swaps_hist_dez18['maturity'].str.split('X').str[1] #getting the maturity of the fra
swaps_hist_dez18.loc[swaps_hist_dez18["instrument"] == "FRA", "maturity"] = swaps_hist_dez18["maturity_fra"] #getting the new maturity
swaps_hist_dez18["year_frac"] = swaps_hist_dez18["maturity"].apply(extract_number)
swaps_hist_dez18.set_index('index',inplace=True)
forward_hist_dez18=swaps_hist_dez18.loc['FRA 6X12', 'par_rate']
one_year_rate_hist_dez18=(((1+swaps_hist_dez18.loc['Deposit 6M', 'par_rate'])**swaps_hist_dez18.loc['Deposit 6M', 'year_frac'])*(1+forward_hist_dez18)**(1-swaps_hist_dez18.loc['Deposit 6M', 'year_frac']))-1
yield_curve_hist_dez18 = swaps_hist_dez18[swaps_hist_dez18.index.str.contains("FRA") == False]
one_year_rate_hist_dez18 = pd.DataFrame([[one_year_rate_hist_dez18,'-','1Y','-',1.0]],
    index=['Swap 1Y'],
    columns=['par_rate', 'instrument','maturity','maturity_fra','year_frac'])
yield_curve_hist_dez18=yield_curve_hist_dez18.append(one_year_rate_hist_dez18)
yield_curve_hist_dez18.set_index('year_frac',inplace=True)
yield_curve_hist_dez18.loc[0.5, 'df'] = 1 / (1 + (yield_curve_hist_dez18.loc[0.5, 'par_rate']/100) * 0.5)
yield_curve_hist_dez18.loc[0.5, 'zero_rate'] = yield_curve_hist_dez18.loc[0.5, 'par_rate']/100
yield_curve_hist_dez18.loc[1, 'df'] = 1 / (1 + yield_curve_hist_dez18.loc[1, 'par_rate']/100 * 1)
yield_curve_hist_dez18.loc[1, 'zero_rate'] = yield_curve_hist_dez18.loc[1, 'par_rate']/100
yield_curve_hist_dez18=yield_curve_hist_dez18.sort_index()
yield_curve_hist_dez18["Tenor"]= yield_curve_hist_dez18.index
yield_curve_hist_dez18['maturity'] = [b+relativedelta(months=+6)] + [date(b.year +int(n), b.month, b.day) for n in yield_curve_hist_dez18.index[1:]]
for n in range(0,len(yield_curve_hist_dez18['maturity'])):
    yield_curve_hist_dez18.iloc[n,2]=date(yield_curve_hist_dez18.iloc[n,2].year,yield_curve_hist_dez18.iloc[n,2].month,yield_curve_hist_dez18.iloc[n,2].day)
yield_curve_hist_dez18['days'] = yield_curve_hist_dez18.maturity.apply(lambda x: (x - date(b.year,b.month,b.day)).days)
idx_hist=0
row_hist=0
for idx_hist, row_hist in yield_curve_hist_dez18.loc[2:].iterrows():
    prev_dfs = yield_curve_hist_dez18.loc[1:idx_hist-1].df.sum()
    yield_curve_hist_dez18.loc[idx_hist, 'df'] = (1 - row_hist.par_rate/100 * prev_dfs) / (1+row_hist.par_rate/100)
for a in yield_curve_hist_dez18.index[2:]:
    yield_curve_hist_dez18.loc[a,'zero_rate']=((1/yield_curve_hist_dez18.loc[a,'df'])**(1/a))-1
b = date(b.year,b.month,b.day)

pd.options.display.float_format = '{:,.5f}'.format
float_leg_hist_dez18 = pd.DataFrame({'notional': swap_notional, 'start': loan_dates[:-1], 'end': loan_dates[1:]})
float_leg_hist_dez18['dcf'] = float_leg_hist_dez18.apply(lambda x: (x.end - x.start).days / 360, axis=1)
float_leg_hist_dez18['fwd'] = float_leg_hist_dez18.apply(lambda row: getFwd(row.start, row.end), axis=1)
float_leg_hist_dez18.loc[0, 'fwd'] = yield_curve_hist_dez18.loc[0.5, 'par_rate'] / 100
float_leg_hist_dez18['cf'] = float_leg_hist_dez18.notional * float_leg_hist_dez18.dcf * float_leg_hist_dez18.fwd
float_leg_hist_dez18['df'] = float_leg_hist_dez18.end.apply(getDF)
float_leg_hist_dez18['pv'] = float_leg_hist_dez18.cf * float_leg_hist_dez18.df
for n in range(0, len(float_leg_hist_dez18['notional'])):
    if  b>=float_leg_hist_dez18.iloc[n, 2]:
        float_leg_hist_dez18.iloc[n, 7]=0
float_leg_hist_pv_dez18 = float_leg_hist_dez18.pv.sum()


fixed_leg_hist_dez18 = pd.DataFrame({'notional': swap_notional, 'start': loan_dates[::1][:-1], 'end': loan_dates[::1][1:]})
fixed_leg_hist_dez18['dcf'] = 1
fixed_leg_hist_dez18['df'] = fixed_leg_hist_dez18.end.apply(getDF)
fixed_leg_hist_dez18['rate'] = fixed_rate
fixed_leg_hist_dez18['cashflow'] = fixed_leg_hist_dez18.notional * fixed_leg_hist_dez18.dcf * fixed_leg_hist_dez18.rate
fixed_leg_hist_dez18['pv'] = fixed_leg_hist_dez18.cashflow * fixed_leg_hist_dez18.df
for n in range(0, len(fixed_leg_hist_dez18['notional'])):
    if  b>=fixed_leg_hist_dez18.iloc[n, 2]:
        fixed_leg_hist_dez18.iloc[n, 7]=0
fixed_leg_hist_pv_dez18 = fixed_leg_hist_dez18.pv.sum()

mk_value_hist_dez18=fixed_leg_hist_pv_dez18-float_leg_hist_pv_dez18
print("Market values is ", mk_value_hist_dez18)

Market values is  32814.26643319161


In [81]:
#2g)
float_leg_hist_dez18

,notional,start,end,dcf,fwd,cf,df,pv
0,"1,748,653.15000",2007-01-19,2007-07-19,0.50278,-0.00520,"-4,571.75651",0.98097,0.00000
1,"1,748,653.15000",2007-07-19,2008-01-21,0.51667,0.03985,"36,003.43733",0.96118,0.00000
2,"1,748,653.15000",2008-01-21,2008-07-21,0.50556,0.04169,"36,852.68842",0.94134,0.00000
3,"1,748,653.15000",2008-07-21,2009-01-19,0.50556,0.04258,"37,646.07566",0.92150,0.00000
4,"1,748,653.15000",2009-01-19,2009-07-20,0.50556,0.04038,"35,698.16367",0.90307,0.00000
5,"1,748,653.15000",2009-07-20,2010-01-19,0.50833,0.04123,"36,646.54485",0.88453,0.00000
6,"1,748,653.15000",2010-01-19,2010-07-19,0.50278,0.04043,"35,542.35504",0.86691,0.00000
7,"1,748,653.15000",2010-07-19,2011-01-19,0.51111,0.04128,"36,893.77046",0.84900,0.00000
8,"1,748,653.15000",2011-01-19,2011-07-19,0.50278,0.04080,"35,868.12115",0.83193,0.00000
9,"1,748,653.15000",2011-07-19,2012-01-19,0.51111,0.04167,"37,239.12433",0.81458,0.00000


In [82]:
#2g
fixed_leg_hist_dez18

,notional,start,end,dcf,df,rate,cashflow,pv
0,"1,748,653.15000",2007-01-19,2007-07-19,1,0.98097,0.01936,"33,850.14008",0.00000
1,"1,748,653.15000",2007-07-19,2008-01-21,1,0.96118,0.01936,"33,850.14008",0.00000
2,"1,748,653.15000",2008-01-21,2008-07-21,1,0.94134,0.01936,"33,850.14008",0.00000
3,"1,748,653.15000",2008-07-21,2009-01-19,1,0.92150,0.01936,"33,850.14008",0.00000
4,"1,748,653.15000",2009-01-19,2009-07-20,1,0.90307,0.01936,"33,850.14008",0.00000
5,"1,748,653.15000",2009-07-20,2010-01-19,1,0.88453,0.01936,"33,850.14008",0.00000
6,"1,748,653.15000",2010-01-19,2010-07-19,1,0.86691,0.01936,"33,850.14008",0.00000
7,"1,748,653.15000",2010-07-19,2011-01-19,1,0.84900,0.01936,"33,850.14008",0.00000
8,"1,748,653.15000",2011-01-19,2011-07-19,1,0.83193,0.01936,"33,850.14008",0.00000
9,"1,748,653.15000",2011-07-19,2012-01-19,1,0.81458,0.01936,"33,850.14008",0.00000
